Run: docker compose -f graph-docker-compose.yml up -d

# Design - dobre praktyki

### 1.  Modeluj dane jako graf, nie jak relacyjną bazę
- Nie przenoś bezpośrednio tabel i JOIN-ów do grafu.
- Skup się na relacjach jako pierwszorzędnych elementach – to właśnie one niosą najwięcej wartości.
- Węzły to obiekty (np. User, Product); a relacje to zależności (:BOUGHT, :FRIENDS_WITH).

### 2. Używaj właściwych etykiet (labels) i typów relacji
- Każdy węzeł powinien mieć co najmniej jedną etykietę (:Person, :Movie); aby efektywnie filtrować.
- Typ relacji (:LIKES, :KNOWS, :WORKS_AT) powinien być jednoznaczny – nie używaj zbyt ogólnych nazw.

### 3. Indeksuj pola używane w wyszukiwaniu
- Indeksy działają tylko na właściwościach węzłów – np. CREATE INDEX FOR (u:User) ON (u.username).
- Neo4j sam używa indeksów w zapytaniach typu MATCH (u:User {username: "jan"}).

### 4. Unikaj tworzenia tzw. super-węzłów
- Węzły z tysiącami relacji (np. :Tag, :Category) mogą spowalniać zapytania.
- Rozważ segmentację relacji lub dodatkowe właściwości filtrowania (np. czas, typ).

### 5. Uprość model – mniej a lepiej
- Lepsze są prostsze zapytania i krótsze ścieżki niż złożone relacje.
- Unikaj „zagnieżdżania” danych we właściwościach – co może być w grafie, niech będzie w grafie.

### 6. Projektuj pod konkretne zapytania (query-driven modeling)
- Model dopasowuj do tego, co będą najczęściej robić użytkownicy/system.
- Testuj zapytania z użyciem EXPLAIN i PROFILE, aby zobaczyć, jak Neo4j wykonuje je wewnętrznie.


In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [2]:
# Połączenie z Neo4j
URI = "bolt://neo4j_nosql_lab:7687"
USERNAME = "neo4j"
PASSWORD = "test1234"  # upewnij się, że to pasuje do graph-docker-compose.yml

In [4]:
q_list = [
        # // CZYSZCZENIE
        "MATCH(n) DETACH DELETE n;"

        # // Węzły Person
        , "CREATE (:Person {name: 'Ala Nowak', username: 'ala25', age: 25});"
        , "CREATE (:Person {name: 'Bartek Kowal', username: 'bartek_k', age: 30});"
        , "CREATE (:Person {name: 'Cezary Lis', username: 'czarek', age: 28});"
        , "CREATE (:Person {name: 'Daria Wiśniewska', username: 'daria', age: 26});"
        , "CREATE (:Person {name: 'Emil Mazur', username: 'emil92', age: 27});"
        , "CREATE (:Person {name: 'Felix Kowalski', username: 'felix', age: 29});"
        , "CREATE (:Person {name: 'Fiona Nowak', username: 'fiona', age: 30});"
        , "CREATE (:Person {name: 'Agata Wiśniewska', username: 'agata26', age: 26});"
        , "CREATE (:Person {name: 'Maria Nowak', username: 'maria31', age: 31});"
        , "CREATE (:Person {name: 'Michał Lisiecki', username: 'mich90', age: 28});"
        , "CREATE (:Person {name: 'Robert Kwiecinski', username: 'robkw21', age: 21});"
        , "CREATE (:Person {name: 'Norbert Smuga', username: 'smuga1', age: 38});"
        , "CREATE (:Person {name: 'Grażyna Kowalska', username: 'grazyna1', age: 30});"


        # // Węzły Group
        , "CREATE (:Group {name: 'Data Science Club', topic: 'Machine Learning'});"
        , "CREATE (:Group {name: 'Board Gamers', topic: 'Games & Social'});"


        # // Tworzenie indeksów
        , "DROP INDEX idx_person_username IF EXISTS;"
        , "DROP INDEX idx_group_name IF EXISTS;"
        , "CREATE INDEX idx_person_username FOR (p:Person) ON (p.username);"
        , "CREATE INDEX idx_group_name FOR (g:Group) ON (g.name);"


        # // Relacje FRIENDS_WITH
        , "MATCH (a:Person {username: 'ala25'}), (b:Person {username: 'bartek_k'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'bartek_k'}), (b:Person {username: 'czarek'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'czarek'}), (b:Person {username: 'daria'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'daria'}), (b:Person {username: 'emil92'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'emil92'}), (b:Person {username: 'ala25'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'fiona'}), (b:Person {username: 'felix'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'felix'}), (b:Person {username: 'agata26'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'agata26'}), (b:Person {username: 'maria31'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'maria31'}), (b:Person {username: 'mich90'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'mich90'}), (b:Person {username: 'robkw21'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'robkw21'}), (b:Person {username: 'grazyna1'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'grazyna1'}), (b:Person {username: 'smuga1'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'smuga1'}), (b:Person {username: 'fiona'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'fiona'}), (b:Person {username: 'maria31'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'maria31'}), (b:Person {username: 'fiona'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'mich90'}), (b:Person {username: 'agata26'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'mich90'}), (b:Person {username: 'fiona'}) CREATE (a)-[:FRIENDS_WITH]->(b);"
        , "MATCH (a:Person {username: 'fiona'}), (b:Person {username: 'mich90'}) CREATE (a)-[:FRIENDS_WITH]->(b);"


        # // Relacje MEMBER_OF
        , "MATCH (p:Person {username: 'ala25'}), (g:Group {name: 'Data Science Club'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'bartek_k'}), (g:Group {name: 'Data Science Club'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'daria'}), (g:Group {name: 'Board Gamers'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'emil92'}), (g:Group {name: 'Board Gamers'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'fiona'}), (g:Group {name: 'Board Gamers'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'felix'}), (g:Group {name: 'Board Gamers'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'maria31'}), (g:Group {name: 'Board Gamers'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'mich90'}), (g:Group {name: 'Data Science Club'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'agata26'}), (g:Group {name: 'Data Science Club'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'robkw21'}), (g:Group {name: 'Data Science Club'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'smuga1'}), (g:Group {name: 'Data Science Club'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'grazyna1'}), (g:Group {name: 'Board Gamers'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'mich90'}), (g:Group {name: 'Board Gamers'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'agata26'}), (g:Group {name: 'Board Gamers'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'robkw21'}), (g:Group {name: 'Board Gamers'}) CREATE (p)-[:MEMBER_OF]->(g);"
        , "MATCH (p:Person {username: 'smuga1'}), (g:Group {name: 'Board Gamers'}) CREATE (p)-[:MEMBER_OF]->(g);"
        

        # // Węzły Post i relacje POSTED + IN_GROUP
        , "CREATE (post:Post {title: 'Intro to ML', content: 'Great workshop on classification', created_at: date('2024-11-10')});"
        , """MATCH (p:Person {username: 'ala25'}), (g:Group {name: 'Data Science Club'})
        MATCH (post:Post {title: 'Intro to ML'})
        CREATE (p)-[:POSTED]->(post)
        CREATE (post)-[:IN_GROUP]->(g);"""

        , "CREATE (post:Post {title: 'ML, Next Steps', content: 'Become a Hero of ML!', created_at: date('2024-12-20')});"
        , """MATCH (p:Person {username: 'mich90'}), (g:Group {name: 'Data Science Club'})
        MATCH (post:Post {title: 'ML, Next Steps'})
        CREATE (p)-[:POSTED]->(post)
        CREATE (post)-[:IN_GROUP]->(g);"""

        , "CREATE (post:Post {title: 'Big Data - How To', content: 'How to get into Big Data World?', created_at: date('2023-08-09')});"
        , """MATCH (p:Person {username: 'mich90'}), (g:Group {name: 'Data Science Club'})
        MATCH (post:Post {title: 'Big Data - How To'})
        CREATE (p)-[:POSTED]->(post)
        CREATE (post)-[:IN_GROUP]->(g);"""

        , "CREATE (post:Post {title: 'Game night!', content: 'Settlers of Catan this Friday?', created_at: date('2025-03-01')});"
        , """MATCH (p:Person {username: 'daria'}), (g:Group {name: 'Board Gamers'})
        MATCH (post:Post {title: 'Game night!'})
        CREATE (p)-[:POSTED]->(post)
        CREATE (post)-[:IN_GROUP]->(g);"""

        , "CREATE (post:Post {title: 'Gamers Life', content: 'The long story of gamers', created_at: date('2025-02-01')});"
        , """MATCH (p:Person {username: 'smuga1'}), (g:Group {name: 'Board Gamers'})
        MATCH (post:Post {title: 'Gamers Life'})
        CREATE (p)-[:POSTED]->(post)
        CREATE (post)-[:IN_GROUP]->(g);"""

        , "CREATE (post:Post {title: 'The best of games', content: 'The best of games is a story...', created_at: date('2024-03-01')});"
        , """MATCH (p:Person {username: 'robkw21'}), (g:Group {name: 'Board Gamers'})
        MATCH (post:Post {title: 'The best of games'})
        CREATE (p)-[:POSTED]->(post)
        CREATE (post)-[:IN_GROUP]->(g);"""

        , "CREATE (post:Post {title: 'My first night session', content: 'My first night session - the story of Grażyna', created_at: date('2024-05-11')});"
        , """MATCH (p:Person {username: 'grazyna1'}), (g:Group {name: 'Board Gamers'})
        MATCH (post:Post {title: 'My first night session'})
        CREATE (p)-[:POSTED]->(post)
        CREATE (post)-[:IN_GROUP]->(g);"""

        , "CREATE (post:Post {title: 'Call of Duty - Tips and Tricks', content: 'Call of Duty - Masters tips and tricks', created_at: date('2025-02-21')});"
        , """MATCH (p:Person {username: 'felix'}), (g:Group {name: 'Board Gamers'})
        MATCH (post:Post {title: 'Call of Duty - Tips and Tricks'})
        CREATE (p)-[:POSTED]->(post)
        CREATE (post)-[:IN_GROUP]->(g);"""
]

### Optymalizacje zastosowane w datasetcie
- Użycie indeksów na username i name dla szybkiego wyszukiwania.
- Unikanie superwęzłów – grupy mają tylko kilku członków, posty są powiązane z grupą, nie wszystkimi użytkownikami.
- Semantycznie jasne relacje (POSTED, IN_GROUP, MEMBER_OF, FRIENDS_WITH).
- Podział informacji – post nie zawiera author_id jako właściwości, tylko osobną relację.
- Model zgodny z query-driven design – łatwo znaleźć:
    - posty użytkownika w grupie,
    - członków grupy,
    - znajomych znajomych.

In [5]:
def run_cypher(query: str, driver):
    with driver.session() as session:
        result = session.run(query)
        records = list(result)
        if not records:
            print("🔍 Brak wyników.")
            return
        # Przekształcenie do DataFrame
        df = pd.DataFrame([r.data() for r in records])
        return df

print("Połączono z Neo4j. Użyj funkcji `run_cypher('TWOJE_ZAPYTANIE')` aby wykonać zapytanie.")

def qexec(query: str):
    driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))
    
    df = run_cypher(query, driver)
    
    driver.close()
    
    return df

Połączono z Neo4j. Użyj funkcji `run_cypher('TWOJE_ZAPYTANIE')` aby wykonać zapytanie.


In [ ]:
for q in q_list:
    print(q)
    qexec(q)

### Ćwiczenia

In [ ]:
# 1. Kto ma najwięcej znajomych?
# - Wyświetl użytkowników i liczbę ich znajomych, posortowanych malejąco.

q = """
...
"""

qexec(q)

In [ ]:
# 2. Kto jest najaktywniejszy w grupach (najwięcej postów)?

q = """
...
"""

qexec(q)

In [ ]:
# 3. Jakie grupy są najpopularniejsze (najwięcej członków)?

q = """
...
"""

qexec(q)

In [ ]:
# 4. Jacy użytkownicy są w tych samych grupach co Ala?

q = """
...
"""

qexec(q)

In [ ]:
# 5. Kto może znać kogo (rekomendacje znajomości przez wspólnych znajomych)?

q = """
...
"""

qexec(q)

In [ ]:
# 6. Średnia liczba znajomych na użytkownika?

q = """
...
"""

qexec(q)

In [ ]:
# 7. Najnowsze posty z każdej grupy

q = """
...
"""

qexec(q)

In [ ]:
# 8. Kto publikuje najwięcej w konkretnej grupie (Board Gamers)?

q = """
...
"""

qexec(q)